# JupyterLIvePlot basic usage

## background

It is not always known at runtime, which data pops up in IOT publish subscriber systems. When it comes plotting these sensor values in a live chart, a flexible solution, which is capable of dynamically add and delete line glyphs is necessary for this task. The class **JupyterLivePlot** is a simple way to plot emerging data, just by passing a dictionary containing the data desired to plot. This notebook shows the usage of this class.

## basic usage

First include the necessary libaries and create an object of a plot. An empty plot will be rendered, yet. This cell is the place where all data will afterwards.

In [1]:
import dynamic_live_plot.jupyter_live_plot as jlp

my_plot = jlp.JupyterLivePlot(plot_width=800, plot_height=410)

Loading BokehJS ...

Create plot
- width: 800 px
- height: 410 px


del line dummy
dummy_glyph


Member **width_d** is the width of the plot, **width_drop** is the number of samples, after which empty plots are deleted.

In [2]:
my_plot.width_d = 500
my_plot.width_drop = 100

To update the plot, data in the form of a dictionary is necessary. Every key in this dictionary corresponds to an individual line plot. The y-values itself are provided by numpy arrays of at least one element. Note that all of these arrays must have the same width. If a line plot is supposed to show empty data, provide np.nan.

To provide data for test purposes, the class **DataFactory** is provided in this scope. The argument *n* in the constructor is the number of the line plots desired to be created. Five plots is the maximum number of plots, that can be created. It provides dictionaries containing random functions. To provide corresponding values for a dedicated x value is the purpose of the class method **produce_data**.

In [3]:
import dynamic_live_plot.data_factory as df
data_generator=df.DataFactory(n=4)
new_data = data_generator.produce_data(0)
print( new_data )

Shuffle all lists, this way iterating over
them has the same result as random choice.
{'magnetic_field': array([2.68845567]), 'temperature': array([-3.50095312]), 'pressure': array([2.67110637]), 'humidity': array([3.60430115])}


To push a column of data into the plot use the method **push_data**. As mentioned above, this is done by using a dictionary.

In [4]:
# first column
new_data = data_generator.produce_data(0)
my_plot.push_data( new_data )
# second column
new_data = data_generator.produce_data(0)
my_plot.push_data( new_data )

In [8]:
import time

data_generator=df.DataFactory(n=3)

dt = 0.01
for i in range(200):
    my_plot.push_data( data_generator.produce_data(my_plot._x) )
    time.sleep(dt)
    
data_generator=df.DataFactory(n=5)  

for i in range(200):
    my_plot.push_data( data_generator.produce_data(my_plot._x) )
    time.sleep(dt)

data_generator=df.DataFactory(n=4)

for i in range(200):
    my_plot.push_data( data_generator.produce_data(my_plot._x) )
    time.sleep(dt)
    
data_generator=df.DataFactory(n=5)

for i in range(200):
    my_plot.push_data( data_generator.produce_data(my_plot._x) )
    time.sleep(dt)

Shuffle all lists, this way iterating over
them has the same result as random choice.


ValueError: stream(...) only supports 1d sequences, got ndarray with size (1, 3)

## non-blocking periodic execution

To be able to do stuff in a nonblocking manner a function can be called periodically after a period of time. This is very similar to the feature of periodic callbacks in Bokeh. A new thread is created for this purpose. Subsequent cells are not blocked and code can be executed, while the callback might be used to retreive and plot data in a concurrent manner.

In [8]:
my_plot._x

array([107. , 107.1])

In [7]:
def cb():
    #my_plot.push_data( data_generator.produce_data(my_plot._x) )
    my_plot.push_data( {"y":[0], "z":[3]} )

my_plot.add_periodic_callback(cb, 10)

Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\jos39406\AppData\Local\Continuum\anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\jos39406\AppData\Local\Continuum\anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jos39406\AppData\Local\Continuum\anaconda3\lib\site-packages\dynamic_live_plot-0.0.2-py3.7.egg\dynamic_live_plot\jupyter_live_plot.py", line 138, in blocking_task
    self.periodic_callback()
  File "<ipython-input-7-bc5a13c74618>", line 3, in cb
    my_plot.push_data( {"y":[0], "z":[3]} )
  File "C:\Users\jos39406\AppData\Local\Continuum\anaconda3\lib\site-packages\dynamic_live_plot-0.0.2-py3.7.egg\dynamic_live_plot\jupyter_live_plot.py", line 183, in push_data
    self.cds.stream( data, self.width_d )
  File "C:\Users\jos39406\AppData\Local\Continuum\anaconda3\lib\site-packages\bokeh\models\sources.py", line 404, in stream
    self._stream(ne

In [10]:
print("Subsequent cells are not blocked from executing, while plot is still running.")

Subsequent cells are not blocked from executing, while plot is still running.


The thread created by **add_periodic_callback** can be deleted any time.

In [6]:
my_plot.remove_periodic_callback()

stop thread
exit.


Todo: Create widgets to subscribe data from an MQTT broker.